In [ ]:
!nvidia-smi


## 1. Confirm the accelerator

Choose `Runtime → Change runtime type` and select **GPU** before running anything else.

If this cell raises an error, go back to `Runtime → Change runtime type`, pick **GPU** and save.


In [3]:
import subprocess

try:
    subprocess.run(['nvidia-smi'], check=True)
except Exception as exc:
    raise RuntimeError(
        'GPU not detected. In Colab, open Runtime → Change runtime type, select GPU (or TPU if GPUs are unavailable), save, then rerun this cell.'
    ) from exc


RuntimeError: GPU not detected. In Colab, open Runtime → Change runtime type, select GPU (or TPU if GPUs are unavailable), save, then rerun this cell.

# Wan2GP on Google Colab

Sets up [Wan2GP](https://github.com/deepbeepmeep/Wan2GP) in a fresh GPU-backed Colab session.

Run the cells in order to prepare the runtime, install dependencies, and launch the Gradio interface. Click on the link in the output from the last cell to launch the app in your browser.

> **Colab VRAM note:** the free tier usually assigns a 15 GB T4 GPU. Most Wan2GP models exceed that budget; the Wan 2.2 TextImage2Video FastWan model works, producing roughly a 5 second 480p clip in about 8 minutes.


## 2. Configure the workspace path

Choose where Wan2GP should be installed. Update `WAN2GP_ROOT` if you prefer a different location.


In [5]:
from pathlib import Path

WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
print(f'Wan2GP will be installed to: {WAN2GP_ROOT}')


Wan2GP will be installed to: /content/Wan2GP


## 3. Download or update Wan2GP

Clone the repository if it is not present yet; otherwise pull the latest changes.


In [6]:
import subprocess

repo_url = 'https://github.com/deepbeepmeep/Wan2GP.git'
if WAN2GP_ROOT.exists():
    print('Repository already exists. Pulling latest changes...')
    subprocess.run(['git', '-C', str(WAN2GP_ROOT), 'pull'], check=True)
else:
    subprocess.run(['git', 'clone', repo_url, str(WAN2GP_ROOT)], check=True)


## 4. Install system dependencies

Install shared libraries needed for video and audio processing. If you see a warning about skipping an extra repository, it is safe to ignore.

In [7]:
import os, subprocess

env = os.environ.copy()
env['DEBIAN_FRONTEND'] = 'noninteractive'

subprocess.run(['sudo', 'apt-get', 'update', '-qq'], check=True, env=env)
subprocess.run([
    'sudo', 'apt-get', 'install', '-y', '--no-install-recommends',
    'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'
], check=True, env=env)


CompletedProcess(args=['sudo', 'apt-get', 'install', '-y', '--no-install-recommends', 'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'], returncode=0)

## 5. Install Python dependencies

Install PyTorch, xformers and Wan2GP's Python packages.

In [8]:
import os, subprocess, sys

env = os.environ.copy()
env.setdefault('DEBIAN_FRONTEND', 'noninteractive')

subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'torch==2.8.0', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu128'], check=True, env=env)
subprocess.run([sys.executable, "-m", "pip", "install", "xformers==0.0.32.post2", "--index-url", "https://download.pytorch.org/whl/cu128"],
               check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(WAN2GP_ROOT / 'requirements.txt')], check=True, env=env)


CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', 'install', '-r', '/content/Wan2GP/requirements.txt'], returncode=0)

## 5b. Force a headless matplotlib backend

Ensure Wan2GP's preprocessing tools use the headless Agg backend so Step 6 launches cleanly in Colab.


In [9]:
from pathlib import Path

# Replace the TkAgg backend with the headless Agg backend if present.
target = WAN2GP_ROOT / 'preprocessing/matanyone/tools/interact_tools.py'
needle = "matplotlib.use('TkAgg')"
replacement = "matplotlib.use('Agg')"

if not target.exists():
    print(f'Skipping: {target} not found.')
else:
    text = target.read_text()
    if replacement in text:
        print('Agg backend already set; no change needed.')
    elif needle in text:
        target.write_text(text.replace(needle, replacement, 1))
        print('Replaced TkAgg with Agg in interact_tools.py.')
    else:
        print('Backend call not found; no change made.')


Replaced TkAgg with Agg in interact_tools.py.


## 6. Launch Wan2GP

Run the Gradio interface. You will find the gradio link in the output. Click on the link to access the UI. Keep the cell running to stay connected; stop it with the square **Stop** button when you are finished.

In [10]:
import os, subprocess, sys, threading, time

env = os.environ.copy()
env.setdefault('WAN_CACHE_DIR', str(WAN2GP_ROOT / 'models'))
cmd = [
    sys.executable,
    '-u',
    'wgp.py',
    '--listen',
    '--server-port', '7860',
    '--share',
    '--profile', '5',
]
print('Launching Wan2GP…')
process = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)
stop_event = threading.Event()

def keepalive():
    while not stop_event.is_set():
        time.sleep(45)
        if stop_event.is_set():
            break
        print('[keepalive] Notebook cell still running…')

keepalive_thread = threading.Thread(target=keepalive, daemon=True)
keepalive_thread.start()

try:
    for line in iter(process.stdout.readline, ''):
        if not line:
            break
        print(line, end='')
except KeyboardInterrupt:
    print('Stopping Wan2GP…')
    process.terminate()
finally:
    stop_event.set()
    process.wait()
    keepalive_thread.join(timeout=1)
    print(f'Wan2GP stopped (return code: {process.returncode}).')


Launching Wan2GP…
2026-02-20 16:20:32.472927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771604432.806868   13805 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771604432.896254   13805 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771604433.523625   13805 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771604433.523793   13805 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771604433.523797   13805 computation_placer.cc:177] comp